# Fine-tune Llama 2 on iSarcasmEval dataset
### This notebook is  inspired by [Fine-tune Llama 2 for sentiment analysis](https://www.kaggle.com/code/lucamassaron/fine-tune-llama-2-for-sentiment-analysis/notebook)  by **Luca Massaron** and [Fine-Tuning LLaMA 2](https://www.datacamp.com/tutorial/fine-tuning-llama-2)

In [ ]:
from huggingface_hub import login, notebook_login

notebook_login()
# login('[YOUR HF API KEY]')

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install -q -U "torch==2.1.2" tensorboard
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
!pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
# import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 19.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import re

# DA with Multi-modal Sarcasm Data (10k)

## Preparing Data

In [ ]:
!git clone https://github.com/iabufarha/iSarcasmEval.git
!git clone https://github.com/headacheboy/data-of-multimodal-sarcasm-detection.git

Cloning into 'iSarcasmEval'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 72 (delta 25), reused 32 (delta 6), pack-reused 0
Receiving objects: 100% (72/72), 535.16 KiB | 14.46 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Cloning into 'data-of-multimodal-sarcasm-detection'...
remote: Enumerating objects: 68, done.
remote: Total 68 (delta 0), reused 0 (delta 0), pack-reused 68
Receiving objects: 100% (68/68), 11.49 MiB | 26.97 MiB/s, done.
Resolving deltas: 100% (17/17), done.


### replace abbreviation

In [ ]:
#https://stackoverflow.com/questions/57503786/python-replace-abbreviation-in-text
replacers = {'dm': 'direct message',
 'thx': 'thanks',
 'dming': 'direct messaging',
 'dmed': 'direct messaged',
 'plz': 'please',
 'u': 'you',
 'asap': 'as soon as possible',
 '...': '',
 '. . .': '',
 'r': 'are',
 'U':'You',
 'idk':"i don't know",
 'omg':'oh my god',
 'sry':'sorry',
  'fb':'facebook',
 }

replace_abb = lambda x: ' '.join([replacers.get(e, e) for e in x])


### replace USERNAME

In [ ]:
replace_username = lambda x: re.sub('@[^\s]+','<user>',x)

# replace_hashtag = lambda x: re.sub('#[^\s]+','<TAG>',x)

replace_link = lambda x: re.sub(r'http?s?://\S+', '<url>', x)

### demojize

In [ ]:
! pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from emoji import emojize, demojize

def replace_emoji(text):
  demojize(text)#delimiters=(":", ":")

### process

### Load external 10k dataset, then combine with the original dataset

In [ ]:
def process(text):
    text = replace_abb(str(text).split())
    text = replace_username(text)
    # text = replace_hashtag(text)
    text = replace_link(text)
    # text = replace_emoji(text)
    return text

In [ ]:
import re
import random
random.seed(42)

In [ ]:
text_file = open("/content/data-of-multimodal-sarcasm-detection/text/train.txt", "r")
lines = text_file.readlines()

# string representation of list to list using ast.literal_eval()
import ast

data_list = []

one = 0
zero = 1

# Converting string to list
for i,line in enumerate(lines):
  res = ast.literal_eval(line)
  data_list.append(res)
  if i == 10000:
    break


X_2 = pd.DataFrame(data_list)

In [ ]:
X_2.rename(columns = { '1': 'tweet', '2': 'sarcastic' }, inplace=True)

In [ ]:
X_3 = X_2.rename(columns = {1: 'tweet', 2: 'sarcastic' })
X_3 = X_3[['tweet','sarcastic']]

In [ ]:
X_3['tweet'] = X_3['tweet'].apply(process)

In [ ]:
X_3

,tweet,sarcastic
0,most # funny quotes : 21 snarky and # funny qu...,1
1,spurs # creativethinking ! <url>,1
2,<user> thanks for showing up for our appointme...,1
3,only a hardcore fan of sir jonny sins will get...,1
4,haha . # lol,1
...,...,...
9996,"barely a teenager , cj collins is now part of ...",0
9997,excited to have the opportunity to compete at ...,0
9998,i 'm in a <user> campaign and that 's wild to ...,0
9999,the boy found this view particularly enticing ...,0


In [ ]:
filename ='/content/iSarcasmEval/train/train.En.csv'
X = pd.read_csv(filename,
                 usecols=["tweet", "sarcastic"],
                 encoding="utf-8", encoding_errors="replace")



In [ ]:
X['tweet'] = X['tweet'].apply(process)
X

,tweet,sarcastic
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,"<user> <user> <user> I did too, and I also rep...",1
...,...,...
3463,The population spike in Chicago in 9 months is...,0
3464,You'd think in the second to last English clas...,0
3465,I’m finally surfacing after a holiday to Scotl...,0
3466,Couldn't be prouder today. Well done to every ...,0


In [ ]:
for index, row in X.iterrows():
    X.at[index, 'tweet'] = demojize(row['tweet'], delimiters=("[", "]"))

X['tweet'][3]

'Today my pop-pop told me I was not “forced” to go to college [upside-down_face] okay sure sureeee'

In [ ]:
X = pd.concat([X, X_3], ignore_index=True,axis=0)

In [ ]:
def generate_prompt(data_point):
    return f"""
            Determine whether the tweet enclosed in square brackets is sarcastic or non-sarcastic,
            and return the answer as the corresponding label "1" for sarcastic or "0" for non-sarcastic.

            Some words like "really", "never", "actually", etc. can be considered a symbol of sarcasm and express some contradictory and criticized attitudes.

            We also notice that some misspelled words (e.g., "so"->"soooo", "love"->"looove", "sure"->"sureeee") and capitalized words (not located at the beginning of a sentence) can sometimes exaggerate the emotional expression.


            [{data_point["tweet"]}] = {data_point["sarcastic"]}
            """.strip()

X = pd.DataFrame(X.apply(generate_prompt, axis=1),
                       columns=["text"])

X = X.sample(frac=1)

train_size = int(len(X)*0.8)
val_size = len(X)-train_size

X_train = X[:train_size]
X_val = X[train_size:]

In [ ]:
X_train

,text
7240,Determine whether the tweet enclosed in square...
11271,Determine whether the tweet enclosed in square...
5242,Determine whether the tweet enclosed in square...
2457,Determine whether the tweet enclosed in square...
6542,Determine whether the tweet enclosed in square...
...,...
12529,Determine whether the tweet enclosed in square...
10937,Determine whether the tweet enclosed in square...
6778,Determine whether the tweet enclosed in square...
12793,Determine whether the tweet enclosed in square...


In [ ]:
X_val

,text
11281,Determine whether the tweet enclosed in square...
11684,Determine whether the tweet enclosed in square...
1450,Determine whether the tweet enclosed in square...
11983,Determine whether the tweet enclosed in square...
9399,Determine whether the tweet enclosed in square...
...,...
5153,Determine whether the tweet enclosed in square...
10281,Determine whether the tweet enclosed in square...
5547,Determine whether the tweet enclosed in square...
834,Determine whether the tweet enclosed in square...


In [ ]:
test_filename ='/content/iSarcasmEval/test/task_A_En_test.csv'

X_test = pd.read_csv(test_filename,
                 usecols=["text", "sarcastic"],
                #  names = ["tweet", "sarcastic"],
                 encoding="utf-8", encoding_errors="replace")

X_test = X_test.rename(columns={'text':'tweet'})

X_test = X_test.sample(frac=1)

def generate_test_prompt(data_point):
    return f"""
            Determine whether the tweet enclosed in square brackets is sarcastic or non-sarcastic,
            and return the answer as the corresponding label "1" for sarcastic or "0" for non-sarcastic.

            Some words like "really", "never", "actually", etc. can be considered a symbol of sarcasm and express some contradictory and criticized attitudes.

            We also notice that some misspelled words (e.g., "so"->"soooo", "love"->"looove", "sure"->"sureeee") and capitalized words (not located at the beginning of a sentence) can sometimes exaggerate the emotional expression.

            [{data_point["tweet"]}] = """.strip()


y_test = X_test['sarcastic']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

In [ ]:
X_test

,text
619,Determine whether the tweet enclosed in square...
1052,Determine whether the tweet enclosed in square...
696,Determine whether the tweet enclosed in square...
921,Determine whether the tweet enclosed in square...
711,Determine whether the tweet enclosed in square...
...,...
594,Determine whether the tweet enclosed in square...
516,Determine whether the tweet enclosed in square...
854,Determine whether the tweet enclosed in square...
1039,Determine whether the tweet enclosed in square...


## Functions

In [ ]:
def evaluate(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",trust_remote_code=True,)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map=device,
                                              torch_dtype=compute_dtype,
                                              quantization_config=bnb_config,)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "1" in answer:
            y_pred.append(1)
        elif "0" in answer:
            y_pred.append(0)
        else:
            y_pred.append(2)
    return y_pred

## Fine-tuning

In [ ]:
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_val)

In [ ]:
repo_name="llama2-7b-base"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=repo_name,                     # directory to save and repository id
    num_train_epochs=4,                       # number of training epochs
    per_device_train_batch_size=4,            # batch size per device during training
    gradient_accumulation_steps=1,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch",               # save checkpoint every epoch
    save_strategy="epoch",
    push_to_hub=True,
)



In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/10775 [00:00<?, ? examples/s]

Map:   0%|          | 0/2694 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.484700,0.389850
2,0.370700,0.381340
3,0.309000,0.383607
4,0.218600,0.390680


TrainOutput(global_step=10776, training_loss=0.3681521225636293, metrics={'train_runtime': 11603.1794, 'train_samples_per_second': 3.714, 'train_steps_per_second': 0.929, 'total_flos': 3.4733451105460224e+17, 'train_loss': 0.3681521225636293, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.39068010449409485,
 'eval_runtime': 187.449,
 'eval_samples_per_second': 14.372,
 'eval_steps_per_second': 1.798,
 'epoch': 4.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1713029017.0447a27193a6.2227.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/K-kiron/llama2-7b-base/commit/aad5c9251bac4ba4957b2a19ef15de320d04ad6d', commit_message='End of training', commit_description='', oid='aad5c9251bac4ba4957b2a19ef15de320d04ad6d', pr_url=None, pr_revision=None, pr_num=None)

## Evaluate

In [ ]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 1400/1400 [09:17<00:00,  2.51it/s]


In [ ]:
evaluate(y_test.tolist(), y_pred)

Accuracy: 0.841
Accuracy for label 0: 0.848
Accuracy for label 1: 0.800

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90      1200
           1       0.47      0.80      0.59       200

    accuracy                           0.84      1400
   macro avg       0.72      0.82      0.75      1400
weighted avg       0.89      0.84      0.86      1400


Confusion Matrix:
[[1018  182]
 [  40  160]]


# Data Augmentation(EDA)

## EDA

In [ ]:
!pip install nltk

In [ ]:
import random
import re
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('stopwords')
random.seed(1)

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Easy data augmentation techniques for text classification
# Jason Wei and Kai Zou
# https://github.com/jasonwei20/eda_nlp

import random
from random import shuffle
random.seed(1)

#stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our',
			'ours', 'ourselves', 'you', 'your', 'yours',
			'yourself', 'yourselves', 'he', 'him', 'his',
			'himself', 'she', 'her', 'hers', 'herself',
			'it', 'its', 'itself', 'they', 'them', 'their',
			'theirs', 'themselves', 'what', 'which', 'who',
			'whom', 'this', 'that', 'these', 'those', 'am',
			'is', 'are', 'was', 'were', 'be', 'been', 'being',
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at',
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after',
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again',
			'further', 'then', 'once', 'here', 'there', 'when',
			'where', 'why', 'how', 'all', 'any', 'both', 'each',
			'few', 'more', 'most', 'other', 'some', 'such', 'no',
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
			'very', 's', 't', 'can', 'will', 'just', 'don',
			'should', 'now', '']

#cleaning up text
import re
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################

#for the first time you use wordnet
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word):
		for l in syn.lemmas():
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym)
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

def random_deletion(words, p):

	#obviously, if there's only one word, don't delete it
	if len(words) == 1:
		return words

	#randomly delete words with probability p
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	#if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		random_word = new_words[random.randint(0, len(new_words)-1)]
		synonyms = get_synonyms(random_word)
		counter += 1
		if counter >= 10:
			return
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

########################################################################
# main data augmentation function
########################################################################

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=1):

	# sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not '']
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	# augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	#trim so that we have the desired number of augmented sentences
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	#append the original sentence
	# augmented_sentences.append(sentence)

	return augmented_sentences[0]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
original_sentence = "The quick brown fox jumps over the lazy dog"
augmented_sentences = eda(original_sentence)
print(augmented_sentences)

The quick brown discombobulate fox jumps over the lazy dog


In [ ]:
def eda_data_point(data_point):
  return eda(str(data_point["tweet"]))

## process

### To control variables, only load the original dataset

In [ ]:
X_original = pd.read_csv(filename,
                 usecols=["tweet", "sarcastic"],
                 encoding="utf-8", encoding_errors="replace")

In [ ]:
X_original['tweet'] = X_original['tweet'].apply(process)

for index, row in X_original.iterrows():
    X_original.at[index, 'tweet'] = demojize(row['tweet'], delimiters=("[", "]"))

X_original['tweet'][3]

'Today my pop-pop told me I was not “forced” to go to college [upside-down_face] okay sure sureeee'

In [ ]:
# X_original = pd.concat([X_original, X_3], ignore_index=True,axis=0)

In [ ]:
X_original.loc[27].tweet

"Not at all concerning that a man's just been round to fiddle with the boiler and now he's left the carbon monoxide alarm won't stop beeping"

In [ ]:
bt_data_point(X_original.loc[27])

["Pas du tout concernant le fait qu'un homme vient de jouer avec la chaudière et maintenant il a laissé l'alarme de monoxyde de carbone ne cessera pas de bip"]
['Not at all about the fact that a man just played with the boiler and now he left the carbon monoxide alarm will not stop beeping']


['Not at all about the fact that a man just played with the boiler and now he left the carbon monoxide alarm will not stop beeping']

In [ ]:
eda_data_point(X_original.loc[27])

"left field Not at all concerning that a whole man's just been round to fiddle with the boiler and now he's left the carbon monoxide alarm won't stop beeping"

In [ ]:
# X_original = X_original.sample(frac=1)

# train_size = int(len(X_original)*0.8)
# val_size = len(X_original)-train_size

# X_train = X_original[:train_size]
# X_val = X_original[train_size:]

In [ ]:
X_to_augment = X_original.loc[X_train.index]

In [ ]:
X_to_augment.loc[3222].tweet

'One of my workmates got me bubble tea for free bc she works in one of the shops [loudly_crying_face] my heart [loudly_crying_face]'

In [ ]:
X_augmented = X_to_augment.copy()

In [ ]:
X_augmented["EDA"]= X_augmented.apply(eda_data_point, axis=1)

In [ ]:
X_augmented

,tweet,sarcastic,EDA
1602,Less than 5 minutes after getting my phone bac...,0,Less than 5 minutes after camp my phone week a...
805,Love seeing 15 year olds being vaccinated in t...,1,goal Love seeing commencement 15 year olds bei...
864,I get a lot of boy who cried wolf vibes from t...,1,I get a lot of boy blood cried wolf Red from t...
321,im about to just walk into a place and start w...,1,im about to just walk into a billet and start ...
2119,Every time I comment on a TikTok asking for ca...,0,Every time I comment on a TikTok asking for ca...
...,...,...,...
2719,This weekend I hit a delivery biker with a car...,0,This weekend I hit a delivery biker with a car...
3040,This country is screwed let’s be honest,0,This country is screwed let’s be personify honest
2105,I always come up with some pretty interesting ...,0,I always come up with some pretty interest con...
133,<user> <user> <user> <user> <user> Finally som...,1,<user> <user> <user> <user> <user> Finally som...


In [ ]:
X_augmented = X_augmented[X_augmented["tweet"]!=X_augmented["EDA"]][["EDA","sarcastic"]]

In [ ]:
X_augmented_ = X_augmented.rename(columns={'EDA': 'tweet'})

In [ ]:
X_augmented_

,tweet,sarcastic
1602,Less than 5 minutes after camp my phone week a...,0
805,goal Love seeing commencement 15 year olds bei...,1
864,I get a lot of boy blood cried wolf Red from t...,1
321,im about to just walk into a billet and start ...,1
2119,Every time I comment on a TikTok asking for ca...,0
...,...,...
2719,This weekend I hit a delivery biker with a car...,0
3040,This country is screwed let’s be personify honest,0
2105,I always come up with some pretty interest con...,0
133,<user> <user> <user> <user> <user> Finally som...,1


In [ ]:
X_augmented = pd.concat([X_to_augment, X_augmented_], ignore_index=True)

In [ ]:
X_augmented_ = pd.DataFrame(X_augmented.apply(generate_prompt, axis=1),
                       columns=["text"])
X_augmented_

,text
0,Determine whether the tweet enclosed in square...
1,Determine whether the tweet enclosed in square...
2,Determine whether the tweet enclosed in square...
3,Determine whether the tweet enclosed in square...
4,Determine whether the tweet enclosed in square...
...,...
5398,Determine whether the tweet enclosed in square...
5399,Determine whether the tweet enclosed in square...
5400,Determine whether the tweet enclosed in square...
5401,Determine whether the tweet enclosed in square...


In [ ]:
X_val

,text
933,Determine whether the tweet enclosed in square...
928,Determine whether the tweet enclosed in square...
823,Determine whether the tweet enclosed in square...
1531,Determine whether the tweet enclosed in square...
1214,Determine whether the tweet enclosed in square...
...,...
1095,Determine whether the tweet enclosed in square...
1130,Determine whether the tweet enclosed in square...
1294,Determine whether the tweet enclosed in square...
860,Determine whether the tweet enclosed in square...


## Fine-tuning

In [ ]:
train_data_aug = Dataset.from_pandas(X_augmented_)
eval_data = Dataset.from_pandas(X_val)

In [ ]:
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM

# config = PeftConfig.from_pretrained("K-kiron/llama2-7b-base")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = PeftModel.from_pretrained(model, "K-kiron/llama2-7b-base")

In [ ]:
output_dir="llama2-7b-EDA"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=4,                       # number of training epochs
    per_device_train_batch_size=4,            # batch size per device during training
    gradient_accumulation_steps=1,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)



In [ ]:
trainer2 = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data_aug,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/5403 [00:00<?, ? examples/s]

Map:   0%|          | 0/694 [00:00<?, ? examples/s]

In [ ]:
trainer2.train()

Epoch,Training Loss,Validation Loss
1,0.332200,0.429807
2,0.294800,0.436925
3,0.244300,0.459591
4,0.237600,0.483046


TrainOutput(global_step=5404, training_loss=0.421005409285371, metrics={'train_runtime': 5655.4875, 'train_samples_per_second': 3.821, 'train_steps_per_second': 0.956, 'total_flos': 1.7547265229770752e+17, 'train_loss': 0.421005409285371, 'epoch': 4.0})

## Evaluate

In [ ]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 1400/1400 [09:10<00:00,  2.54it/s]


In [ ]:
evaluate(y_test.tolist(), y_pred)

Accuracy: 0.829
Accuracy for label 0: 0.827
Accuracy for label 1: 0.840

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.83      0.89      1200
           1       0.45      0.84      0.58       200

    accuracy                           0.83      1400
   macro avg       0.71      0.83      0.74      1400
weighted avg       0.89      0.83      0.85      1400


Confusion Matrix:
[[992 208]
 [ 32 168]]
